In [ ]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 19.0 MB/s 
     |████████████████████████████████| 168 kB 25.3 MB/s 
     |████████████████████████████████| 182 kB 28.5 MB/s 
     |████████████████████████████████| 62 kB 330 kB/s 
     |████████████████████████████████| 168 kB 14.7 MB/s 
     |████████████████████████████████| 166 kB 14.4 MB/s 
     |████████████████████████████████| 166 kB 5.7 MB/s 
     |████████████████████████████████| 162 kB 58.4 MB/s 
     |████████████████████████████████| 162 kB 17.4 MB/s 
     |████████████████████████████████| 158 kB 25.0 MB/s 
     |████████████████████████████████| 157 kB 51.6 MB/s 
     |████████████████████████████████| 157 kB 55.9 MB/s 
     |████████████████████████████████| 157 kB 36.4 MB/s 
     |████████████████████████████████| 157 kB 52.2 MB/s 
     |████████████████████████████████| 157 kB 49.3 MB/s 
     |████████████████████████████████| 157 kB 23.1 MB/s 
     |████████████████████████████████| 157 kB 11.5 MB/s 
     |███████████

In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!pip install rdkit-pypi -qqq

     |████████████████████████████████| 29.3 MB 1.9 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls ./gdrive

Mounted at /content/gdrive
MyDrive


# New Section

In [13]:
import math, sys, random, os
import numpy as np
import time
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
import json
import gzip
import six
import pandas as pd
import csv

import os


project_root = os.path.dirname('/content/gdrive/MyDrive/scscore-master/')
# project_root = os.path.dirname(os.path.dirname(__file__))

score_scale = 5.0
min_separation = 0.25

FP_len = 1024
FP_rad = 2

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

class SCScorer():
    def __init__(self, score_scale=score_scale):
        self.vars = []
        self.score_scale = score_scale
        self._restored = False

    def restore(self, weight_path=os.path.join(project_root, 'models', 'full_reaxys_model_1024bool', 'model.ckpt-10654.as_numpy.pickle'), FP_rad=FP_rad, FP_len=FP_len):
        self.FP_len = FP_len; self.FP_rad = FP_rad
        self._load_vars(weight_path)
        print('Restored variables from {}'.format(weight_path))

        if 'uint8' in weight_path or 'counts' in weight_path:
            def mol_to_fp(self, mol):
                if mol is None:
                    return np.array((self.FP_len,), dtype=np.uint8)
                fp = AllChem.GetMorganFingerprint(mol, self.FP_rad, useChirality=True) # uitnsparsevect
                fp_folded = np.zeros((self.FP_len,), dtype=np.uint8)
                for k, v in six.iteritems(fp.GetNonzeroElements()):
                    fp_folded[k % self.FP_len] += v
                return np.array(fp_folded)
        else:
            def mol_to_fp(self, mol):
                if mol is None:
                    return np.zeros((self.FP_len,), dtype=np.float32)
                return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, self.FP_rad, nBits=self.FP_len,
                    useChirality=True), dtype=np.bool)
        self.mol_to_fp = mol_to_fp

        self._restored = True
        return self

    def smi_to_fp(self, smi):
        if not smi:
            return np.zeros((self.FP_len,), dtype=np.float32)
        return self.mol_to_fp(self, Chem.MolFromSmiles(smi))

    def apply(self, x):
        if not self._restored:
            raise ValueError('Must restore model weights!')
        # Each pair of vars is a weight and bias term
        for i in range(0, len(self.vars), 2):
            last_layer = (i == len(self.vars)-2)
            W = self.vars[i]
            b = self.vars[i+1]
            x = np.matmul(x, W) + b
            if not last_layer:
                x = x * (x > 0) # ReLU
        x = 1 + (score_scale - 1) * sigmoid(x)
        return x

    def get_score_from_smi(self, smi='', v=False):
        if not smi:
            return ('', 0.)
        fp = np.array((self.smi_to_fp(smi)), dtype=np.float32)
        if sum(fp) == 0:
            if v: print('Could not get fingerprint?')
            cur_score = 0.
        else:
            # Run
            cur_score = self.apply(fp)
            if v: print('Score: {}'.format(cur_score))
        mol = Chem.MolFromSmiles(smi)
        if mol:
            smi = Chem.MolToSmiles(mol, isomericSmiles=True, kekuleSmiles=True)
        else:
            smi = ''
        return (smi, cur_score)

    def _load_vars(self, weight_path):
        if weight_path.endswith('pickle'):
            import cPickle as pickle
            with open(weight_path, 'rb') as fid:
                self.vars = pickle.load(fid)
                self.vars = [x.tolist() for x in self.vars]
        elif weight_path.endswith('json.gz'):
            with gzip.GzipFile(weight_path, 'r') as fin:    # 4. gzip
                json_bytes = fin.read()                      # 3. bytes (i.e. UTF-8)
                json_str = json_bytes.decode('utf-8')            # 2. string (i.e. JSON)
                self.vars = json.loads(json_str)
                self.vars = [np.array(x) for x in self.vars]


model = SCScorer()
model.restore(os.path.join(project_root, 'models', 'full_reaxys_model_1024bool', 'model.ckpt-10654.as_numpy.json.gz'))

smis = ['Cc1ccc2c(-c3cc[nH]c(=Nc4ccc(F)c(F)c4)n3)c(-c3ccccc3)nn2n1', 'Cc1ccc2c(-c3ccnc(Nc4ccc(F)c(F)c4)n3)c(-c3ccccc3)nn2n1']
for smi in smis:
      (smi, sco) = model.get_score_from_smi(smi)
      print('%.4f <--- %s' % (sco, smi))





Restored variables from /content/gdrive/MyDrive/scscore-master/models/full_reaxys_model_1024bool/model.ckpt-10654.as_numpy.json.gz
4.6107 <--- CC1=NN2N=C(C3=CC=CC=C3)C(C3=NC(=NC4=CC=C(F)C(F)=C4)NC=C3)=C2C=C1
4.5362 <--- CC1=NN2N=C(C3=CC=CC=C3)C(C3=CC=NC(NC4=CC=C(F)C(F)=C4)=N3)=C2C=C1


<ipython-input-13-8d468f60b8fa>:52: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  useChirality=True), dtype=np.bool)


In [9]:


with open('/content/gdrive/MyDrive/scscore-master/positive_emols.csv', 'r', encoding="utf-8") as f:

  input_variable = []
  for row in f:
    (row,sco) = model.get_score_from_smi(row)
    input_variable.append(['%.4f , %s' % (sco, row)])
 

with open ('/content/gdrive/MyDrive/scscore-master/emols_SC_test_result.csv','a',newline = '') as csvfile:
      my_writer = csv.writer(csvfile, delimiter = ' ')
      my_writer.writerows(input_variable)

<ipython-input-8-4495753b0747>:52: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  useChirality=True), dtype=np.bool)
